In [272]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import numpy as np
import galois   

In [273]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [348]:
# This script demonstrates the polynomial evaluations with powers of tau
# we see that evaluating a polyomial at tau is the same as computing the inner product of the polynomial coefficients and the powers of tau

tau_testing = GF(20)

# (4x^3 + 2x^2 + 3x + 1) * (x+2)
polynomial = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
print(f"Polynomial: {polynomial}")  # Outputs: 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the polynomial at tau.
p_eval_at_tau = polynomial(tau_testing)
print(f"Polynomial evaluated at tau: {p_eval_at_tau}") # Outputs: 3962
p_eval_at_tau_G1 = multiply(G1, int(p_eval_at_tau))

def powers_of_tau_test(tau, degree):
    powers_of_tau_testing = []
    for i in range(degree+1):
        print(f"i = {i}, tau**i = {tau**i}")
        powers_of_tau_testing.append(multiply(G1, int(tau**i)))
    return powers_of_tau_testing

def compute_powers_of_tau_test(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree + 1):
        tau_power = tau ** i
        tau_power_G1 = multiply(G1, int(tau_power))
        powers_of_tau.append(tau_power_G1)
        print(f"i = {i}, tau**i = {tau_power}, tau_power_G1 = {tau_power_G1}")
    return powers_of_tau


# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau_testing = compute_powers_of_tau_test(tau_testing, polynomial.degree)

# reverse the polynomial coefficients for 4x^4 + 10x^3 + 7x^2 + 7x + 2  to [2, 7, 7, 10, 4]
reversed_polynomial_coeffs = polynomial.coeffs[::-1]
print(f"Reversed polynomial coefficients: {reversed_polynomial_coeffs}")

def inner_product_test(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed polynomial coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

assert inner_product_test(powers_of_tau_testing, reversed_polynomial_coeffs) == p_eval_at_tau_G1, "doesn't match"

Polynomial: 4x^4 + 10x^3 + 7x^2 + 7x + 2
Polynomial evaluated at tau: 722942
i = 0, tau**i = 1, tau_power_G1 = (1, 2)
i = 1, tau**i = 20, tau_power_G1 = (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992)
i = 2, tau**i = 400, tau_power_G1 = (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714)
i = 3, tau**i = 8000, tau_power_G1 = (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677)
i = 4, tau**i = 160000, tau_power_G1 = (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)
Reversed polynomial coefficients: [ 2  7  7 10  4]


In [378]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [GF(curve_order-2),1,0,3,0,GF(curve_order-1),0,GF(curve_order-2)] # 113 - 2 = 111
    ]))

# in R1CS # m = number of columns # n = number of rows
r1cs_m = L.shape[1]
r1cs_n = L.shape[0]
print(f"m={r1cs_m}, n={r1cs_n}")

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

m=8, n=5
witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [396]:
# R1CS => QAP
# in QAP m = number of polynomials. n = degree of target polynomial
tau = GF(20)
target_coeffs = GF(np.array([1,2,3,4,5]))
tx_poly = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_polys = matrix_to_vector_of_polynomials(L)
V_vector_of_polys = matrix_to_vector_of_polynomials(R)
W_vector_of_polys = matrix_to_vector_of_polynomials(O)

qap_m = len(U_vector_of_polys)
qap_n = tx_poly.degree # not sure if this is right

# these 2 functions demonstrate that we can either
# 1) multiply polynomials and witness together to 1 polynomial, then evaluate at tau
def aggregate_vector_of_polynomials_with_witness(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        sum += vector_of_poly[i]*witness[i]
    return sum

# 2) evaluate each polynomial at tau, then multiply by witness
def aggregate_vector_of_polynomials_with_witness_at_tau(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        sum += vector_of_poly[i](tau) * witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua_poly = aggregate_vector_of_polynomials_with_witness(U_vector_of_polys, witness, verbose=True)
Va_poly = aggregate_vector_of_polynomials_with_witness(V_vector_of_polys, witness)
Wa_poly = aggregate_vector_of_polynomials_with_witness(W_vector_of_polys, witness)

# We can now create h(x)t(x)
# we test if the polynomial is the same
hx_poly, remainder = divmod(((Ua_poly*Va_poly) - Wa_poly), tx_poly)
assert remainder == 0, "remainder is not 0"
Ua_poly * Va_poly == Wa_poly + hx_poly * tx_poly
print(f"lhs = rhs so they are the same")

# then test if the polynomial evaluated at tau is the same (schwartz zippel)
tau = GF(9)
Ua_poly(tau) * Va_poly(tau) == Wa_poly(tau) + hx_poly(tau) * tx_poly(tau)
print(f"when evaluated at random number tau, they are the same")

scaling witness 1 by 0
scaling witness 97 by 0
scaling witness 5 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x
scaling witness 10 by 16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10
scaling witness 25 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 18240202393199396018538671454381062573790303667013361953081836822146507079683x^3 + 18240202393199396018538671454381062573790303667013361953081836822146507079671x^2 + 364804047863987920370773429087621251475806073340267239061

In [398]:
# We use trusted setup to hide polynomial coeffs of U, V, W, h, t
from functools import reduce
tau = GF(20)


# Trusted Setup
# Generate 1) Powers of tau G1, G2. 2) Powers of tau for target polynomial G1. 3) target polynomial evaluated at tau
powers_of_tau_G1_points = [multiply(G1, int(tau) ** i) for i in range(0,qap_n)]
powers_of_tau_G2_points = [multiply(G2, int(tau) ** i) for i in range(0,qap_n)]
tx_powers_of_tau_G1_points = [multiply(G1, int(tau) ** i) for i in range(0,tx_poly.degree)]
t_of_tau_scalar = tx_poly(tau)
tx_powers_of_tau_G1_eval_at_tau = [multiply(tx_powers_of_tau_G1_points[i], int(t_of_tau_scalar)) for i in range(0, len(tx_powers_of_tau_G1_points))]

# Quick sanity test - this is not encrypted and wouldn't happen in real life
Ua_tau_scalar = Ua_poly(tau)
Va_tau_scalar = Va_poly(tau)
Wa_tau_scalar = Wa_poly(tau)
hx_tau_scalar = hx_poly(tau)
tx_tau_scalar = tx_poly(tau)
print(f"Ua(tau) = {Ua_tau_scalar}")
print(f"Va(tau) = {Va_tau_scalar}")
print(f"Wa(tau) = {Wa_tau_scalar}")
print(f"hx(tau) = {hx_tau_scalar}")
print(f"tx(tau) = {tx_tau_scalar}")
print(f"ht= {hx_tau_scalar * tx_tau_scalar}")

assert Ua_tau_scalar * Va_tau_scalar == Wa_tau_scalar + hx_tau_scalar * tx_tau_scalar, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

def inner_product_iterable(powers_of_tau, coeffs):
    # print(len(powers_of_tau_G1), " eq? ", len(coeffs))
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        # print(f"Iteration: {i}, Coeff: {coeff}, Point: {point}")
        result = add(result, multiply(point, int(coeff)))
    return result
 
Ua_tau_G1_point = inner_product_iterable(powers_of_tau_G1_points, Ua_poly.coeffs[::-1])
Va_tau_G2_point = inner_product_iterable(powers_of_tau_G2_points, Va_poly.coeffs[::-1])
Wa_tau_G1_point = inner_product_iterable(powers_of_tau_G1_points, Wa_poly.coeffs[::-1])

# we use from [o:hx.degree + 1] because we need to cover all hx coeffs, not just exponentiated
ht_tau_G1_point = inner_product_iterable(tx_powers_of_tau_G1_eval_at_tau[0:hx_poly.degree + 1], hx_poly.coeffs[::-1])

AB = pairing(Va_tau_G2_point, neg(Ua_tau_G1_point))
CD = pairing(G2, add(Wa_tau_G1_point, ht_tau_G1_point))
final_exponentiate(AB * CD) == FQ12.one()

Ua(tau) = 21888242871839275222246405745257275088548364400416034343698204186575806479342
Va(tau) = 398150
Wa(tau) = 21888242871839275222246405745257275088548364400416034343698204186575781150367
hx(tau) = 8512094450159718141984713345377829201102141711272902244771523850335036061883
tx(tau) = 1395360
ht= 21888242871839275222246405745257275088548364400416034343698204185773055949617


True

In [403]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha_scalar = GF(2)
beta_scalar = GF(3)
AlphaG1_point = multiply(G1, int(alpha_scalar))
BetaG2_point = multiply(G2, int(beta_scalar))

# # reminder
# # qap_m = len(U_vector_of_poly_x). qap_n = target_polynomial.degree # not sure if this is right

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
# Trusted Setup completes this with unencrypted tau, alpha, beta values
# Powers of tau for C 
powers_of_tau_C_G1_points = []
for i in range(0, qap_m):
    iterator_u_scalar = U_vector_of_polys[i](tau)
    iterator_v_scalar = V_vector_of_polys[i](tau)
    iterator_w_scalar = W_vector_of_polys[i](tau)
    sum_scalar = int(beta_scalar * iterator_u_scalar + alpha_scalar * iterator_v_scalar + iterator_w_scalar) * int(witness[i])
    print(f"sum polynomial for iterator {i} = {sum_scalar}")
    powers_of_tau_C_G1_points.append(multiply(G1, sum_scalar))
    # print(f"current powers of tau G1 = {powers_of_tau_C_G1}")
print(f"powers of tau C G1 points = {powers_of_tau_C_G1_points}")

def inner_product_iterable_test(powers_of_tau, coeffs):
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        result = add(result, multiply(point, int(coeff)))
    return result

def powers_of_tau_innerproduct_poly_i(powers_of_tau, vector_of_polys, witness):
    accumulator = Z1
    for i in range(0, len(vector_of_polys)):
        iterator_point = inner_product_iterable_test(powers_of_tau, vector_of_polys[i].coeffs[::-1])
        accumulator = add(accumulator, multiply(iterator_point, int(witness[i])))
    return accumulator    

A_G1_point = powers_of_tau_innerproduct_poly_i(powers_of_tau_G1_points, U_vector_of_polys, witness)
B_G2_point = powers_of_tau_innerproduct_poly_i(powers_of_tau_G2_points, V_vector_of_polys, witness)

A_G1_point = add(A_G1_point, AlphaG1_point)
B_G2_point = add(B_G2_point, BetaG2_point)

C_G1_point = Z1
for i in range(len(powers_of_tau_C_G1_points)):
    C_G1_point = add(C_G1_point, powers_of_tau_C_G1_points[i])

C_G1_point = add(C_G1_point, ht_tau_G1_point)

print("A_G1_point", A_G1_point)
print("B_G2_point", B_G2_point)
print("C_G1_point", C_G1_point)

AB = pairing(B_G2_point, neg(A_G1_point))
CD = pairing(G2, C_G1_point)
EF = pairing(BetaG2_point, AlphaG1_point)
final_exponentiate(AB * CD * EF) == FQ12.one()

sum polynomial for iterator 0 = 21888242871839275222246405745257275088548364400416034343698204186575808487865
sum polynomial for iterator 1 = 375972
sum polynomial for iterator 2 = 109441214359196376111232028726286375442741822002080171718491020932879042134685
sum polynomial for iterator 3 = 851580
sum polynomial for iterator 4 = 547206071795981880556160143631431877213709110010400858592455104664395211497925
sum polynomial for iterator 5 = 2736030358979909402780800718157159386068545550052004292962275523321976054402000
sum polynomial for iterator 6 = 2827200
sum polynomial for iterator 7 = 27360303589799094027808007181571593860685455500520042929622755233219760591662500
powers of tau C G1 points = [(18412086319910179752494782965811456442455794258224578119780579512176567504706, 10558562959246852549340863372112227129585961702953419993535490858069696891127), (3241640664343904866761662067828845588248524718703942440965736023541316835185, 500216134051332044131722665126354567306139756979318283363

True

In [279]:
# separate public and private inputs
# public input, computed by verifier
public_witness_length = 2
public_input = []

for i in range(public_witness_length):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    public_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

# we now have 2 points representing Bui, Avi, Wvi. Let's squash the witness in and add to 1 point
public_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(public_input, witness[:public_witness_length])), Z1)

# private input, computed by prover. Prover has access to witness and 
private_input = []
for i in range(public_witness_length,len(witness)):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    private_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

private_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(private_input, witness[public_witness_length:])), Z1)

C_new = add(Cprime, ht_tau_G1)

iteration 0 witness:  1
iteration 1 witness:  97
iteration 2 witness:  5
iteration 3 witness:  10
iteration 4 witness:  25
iteration 5 witness:  125
iteration 6 witness:  100
iteration 7 witness:  1250


In [280]:
# Introducing Gamma, Delta
gamma = GF(4)
delta = GF(5)
gamma_inv = np.reciprocal(gamma)
delta_inv = np.reciprocal(delta)

GammaG2 = multiply(G2, int(gamma))
DeltaG2 = multiply(G2, int(delta))

# powers of tau for public input from i = 0 to i = 1 for the 0, 1 pos of [1,out,x,y,...] 
public_witness_length = 2

# public
public_powers_of_tau_G1 = [multiply(point, int(gamma_inv)) for point in powers_of_tau_C_G1[:public_witness_length]]
gamma_pairing = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(public_powers_of_tau_G1, witness[0:public_witness_length])), Z1)

# private
private_powers_of_tau_G1 = [multiply(point, int(delta_inv)) for point in powers_of_tau_C_G1[public_witness_length+1:len(witness)]]
Cprime_delta = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(private_powers_of_tau_G1, witness[public_witness_length:len(witness)])), Z1)


#ht_tau is from i = 0 -> m. Needs to change to l -> m
delta_ht_tau = multiply(ht_tau_G1, int(delta_inv))
C_prime_new = add(Cprime_delta, ht_tau_G1)
C_new = add(Cprime, ht_tau_G1)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(GammaG2, gamma_pairing)
rhs_3 = pairing(DeltaG2, C_prime_new)
rhs = rhs_1 * rhs_2 * rhs_3

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

not verified


In [281]:
import numpy as np
from scipy.interpolate import lagrange
x = np.array([1,2,3])
y = np.array([1,1,0])
print(lagrange(x, y))

      2
-0.5 x + 1.5 x
